In [325]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()


In [326]:
class Linear():
    def __init__(self,inputs,outputs):
        self.weights=0.01*np.random.randn(outputs,inputs)
        self.biases=np.zeros((1,outputs))

    def forward(self,inputs):
        self.inputs=inputs
        self.outputs=np.dot(inputs,self.weights.T)+self.biases
        return self.outputs
    
    def backward(self,dnextinputs):
        self.dweights=np.dot(dnextinputs.T,self.inputs)
        self.dbiases=np.sum(dnextinputs,axis=0,keepdims=True)
        self.dinputs=np.dot(dnextinputs,self.weights)
        return self.dinputs

In [327]:
class Relu():
    def __init__(self) -> None:
        pass
    
    def forward(self,inputs):
        self.inputs=inputs
        self.outputs=np.maximum(0,inputs)
        return self.outputs


    def backward(self,dnextinputs):
        self.dinputs=dnextinputs
        self.dinputs[self.inputs<=0]=0
        return self.dinputs



    

In [328]:
class Softmax():
    def __init__(self) -> None:
        pass

    def forward(self,inputs):
        self.inputs=inputs
        exp=np.exp(inputs -np.max(inputs, axis=1,keepdims=True))
        exp_sum=np.sum(exp,axis=1,keepdims=True)
        self.outputs=exp/exp_sum
        return self.outputs

    def backwards(self,dnextinputs):
        self.dinputs = np.zeros_like(dnextinputs)
        i=0
        for output, dnextinput in zip(self.outputs, dnextinputs):
            output = output.reshape(-1, 1)
            Jacobian= np.diagflat(output) - np.dot(output,output.T)
            self.dinputs[i] = np.dot(Jacobian, dnextinput)
            i+=1
        return self.dinputs


In [333]:
class CrossEntropyLoss():
    def __init__(self) -> None:
        pass
        
    def forward(self,inputs,labels):
        self.labels=labels
        self.inputs=inputs
        epsilon=1e-7
        inputs=np.clip(inputs,epsilon,1-epsilon)
        one_hot=np.eye(len(inputs[0]))[labels]
        self.output=-np.sum(one_hot*np.log(inputs))/one_hot.shape[0]
        return self.output
    
    def backward(self):
        one_hot=np.eye(len(self.inputs[0]))[self.labels]
        self.dinputs=(-one_hot/self.inputs)/one_hot.shape[0]
        return self.dinputs


class Optimizer_Adam():
    def __init__(self,learning_rate):
        self.learning_rate=learning_rate
        self.current_learning_rate=learning_rate
        self.learning_rate_step

In [342]:
a=np.array([2,3,4,1,2])
b=np.array([3,4,7,-1,3])
c=a-b
c.shape[0]


5

In [336]:
X, y = spiral_data(samples=100, classes=3)
X=np.array([[0.        , 0.        ],
 [0.00299556, 0.00964661],
 [0.01288097, 0.01556285],
 [0.02997479, 0.0044481 ],
 [0.03931246 ,0.00932828]])
y=np.array([0, 0 ,2 ,0 ,0])

dense1 = Linear(2, 3)

activation1 =Relu()


dense2 = Linear(3, 3)

activation2 =Softmax()
lossFun=CrossEntropyLoss()



x=dense1.forward(X)
x=activation1.forward(x)
x=dense2.forward(x)
x=activation2.forward(x)
loss=lossFun.forward(x,y)

print("loss",loss)
predictions = np.argmax(x, axis=1)
accuracy = np.mean(predictions==y)
print('acc:', accuracy)


# # Backward pass
z=lossFun.backward()
z=activation2.backwards(z)
z=dense2.backward(z)
z=activation1.backward(z)
z=dense1.backward(z)


# Print gradients
print("d1w",dense1.dweights.T)
print("d1b",dense1.dbiases)
print("d2w",dense2.dweights.T)
print("d2b",dense2.dbiases)

[0 0 2 0 0]
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]
loss 1.0986108779907227
acc: 0.8
d1w [[-8.8799483e-05  1.0494100e-05  5.0043786e-07]
 [ 2.5282593e-06  2.4603557e-06  1.6115614e-06]]
d1b [[-0.00158801 -0.00048641  0.00016706]]
d2w [[-1.2790557e-04  7.4393909e-05  5.3511663e-05]
 [-7.6857086e-06  4.7535004e-06  2.9322084e-06]
 [-6.4063607e-07  3.2031804e-07  3.2031804e-07]]
d2b [[-0.46666607  0.3333333   0.13333276]]


In [343]:
import pandas as pd

In [372]:
dataraw=pd.read_csv("Arrhythmia_MLR/arrhythmia.data",header=None)
import pandas as pd
with open('Arrhythmia_MLR/arrhythmia.names', 'r') as file:
    lines = file.readlines()

# pobranie nazw atrybutów z pliku .names
attribute_names = []
for line in lines:
    if line.startswith('|'):
        attribute_name = line.strip().split(':')[0][1:].strip()
        attribute_names.append(attribute_name)

# wczytanie pliku .data z wykorzystaniem biblioteki pandas
data = pd.read_csv('Arrhythmia_MLR/arrhythmia.data', header=None, delimiter=',', names=attribute_names)

# zapisanie danych do pliku CSV
data.to_csv('nazwa_pliku.csv', index=False)

[('1. Title', 'Cardiac_Arrhythmia_Database'), ('2. Sources', ''), ('3. Past Usage', ''), ('4. Relevant Information', ''), ('5. Number of Instances', '452'), ('6. Number of Attributes', '279'), ('7. Attribute Information', ''), ('8. Missing Attribute Values', "Several.__Distinguished_with_'?'."), ('9. Class Distribution', '')]


In [364]:
data.head(100)

,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
75,0,190,80,91,193,371,174,121,-16,13,64,-2,?,63,0,52,44,0,0,32,0,0,0,0,0,0,0,44,20,36,0,28,0,0,0,0,0,0,52,40,0,0,0,60,0,0,0,0,0,0,52,0,0,0,0,0,0,0,0,0,0,0,0,56,36,0,0,32,0,0,0,0,0,0,48,32,0,0,0,56,0,0,0,0,0,0,80,0,0,0,0,0,0,0,0,0,0,0,0,40,52,0,0,28,0,0,0,0,0,0,0,48,48,0,0,32,0,0,0,0,0,0,0,52,52,0,0,36,0,0,0,0,0,0,0,52,48,0,0,32,0,0,0,0,0,0,0,56,44,0,0,32,0,0,0,0,0,0,-0.2,0.0,6.1,-1.0,0.0,0.0,0.6,2.1,13.6,30.8,0.0,0.0,1.7,-1.0,0.6,0.0,1.3,1.5,3.7,14.5,0.1,-5.2,1.4,0.0,0.0,0.0,0.8,-0.6,-10.7,-15.6,0.4,-3.9,0.0,0.0,0.0,0.0,-0.8,-1.7,-10.1,-22.0,0.0,0.0,5.7,-1.0,0.0,0.0,-0.1,1.2,14.1,22.5,0.0,-2.5,0.8,0.0,0.0,0.0,1.0,0.4,-4.8,-2.7,0.1,-6.0,0.0,0.0,0.0,0.0,-0.8,-0.6,-24.0,-29.7,0.0,0.0,2.0,-6.4,0.0,0.0,0.2,2.9,-12.6,15.2,-0.1,0.0,8.4,-10.0,0.0,0.0,0.6,5.9,-3.9,52.7,-0.3,0.0,15.2,-8.4,0.0,0.0,0.9,5.1,17.7,70.7,-0.4,0.0,13.5,-4.0,0.0,0.0,0.9,3.9,25.5,62.9,-0.3,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,8
56,1,165,64,81,174,401,149,39,25,37,-17,31,?,53,0,48,0,0,0,24,0,0,0,0,0,0,0,64,0,0,0,24,0,0,0,0,0,0,32,24,0,0,0,40,0,0,0,0,0,0,48,0,0,0,0,0,0,0,0,0,0,0,0,44,20,0,0,24,0,0,0,0,0,0,0,60,0,0,0,20,0,0,0,0,0,0,0,24,52,0,0,16,0,0,0,0,0,0,0,32,52,0,0,20,0,0,0,0,0,0,0,44,48,0,0,32,0,0,0,0,0,0,0,48,44,0,0,32,0,0,0,0,0,0,0,48,40,0,0,28,0,0,0,0,0,0,0,48,0,0,0,28,0,0,0,0,0,0,-0.6,0.0,7.2,0.0,0.0,0.0,0.4,1.5,17.2,26.5,0.0,0.0,5.5,0.0,0.0,0.0,0.1,1.7,17.6,29.5,0.3,-1.6,0.9,0.0,0.0,0.0,-0.3,0.4,-1.5,1.3,0.1,-6.4,0.0,0.0,0.0,0.0,-0.3,-1.6,-15.3,-25.5,-0.3,0.0,4.2,-0.9,0.0,0.0,0.4,0.7,8.3,12.3,0.2,0.0,2.2,0.0,0.0,0.0,-0.2,0.8,6.6,11.7,0.4,0.0,1.0,-8.8,0.0,0.0,0.5,-0.6,-21.6,-26.8,0.4,0.0,2.6,-7.9,0.0,0.0,0.8,2.0,-16.4,1.2,0.0,0.0,5.8,-7.7,0.0,0.0,0.9,3.8,-5.7,27.7,-0.2,0.0,9.5,-5.0,0.0,0.0,0.5,2.6,11.8,34.6,-0.4,0.0,11.0,-2.4,0.0,0.0,0.4,2.6,21.6,43.4,-0.5,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,6
54,0,172,95,138,163,386,185,102,96,34,70,66,23,75,0,40,80,0,0,24,0,0,0,0,0,0,20,56,52,0,0,40,0,0,0,0,0,0,28,116,0,0,0,52,0,0,0,0,0,0,52,64,0,0,0,88,0,0,0,0,0,0,0,36,92,0,0,24,0,0,0,0,0,0,0,128,0,0,0,24,0,1,0,0,0,0,0,24,36,76,0,100,0,0,0,0,0,0,0,40,28,60,0,96,0,0,0,0,0,0,0,48,20,56,24,32,0,0,0,0,0,0,0,44,88,0,0,28,0,0,0,0,0,0,0,44,76,0,0,28,0,0,0,0,0,0,0,44,72,0,0,24,0,0,0,0,0,0,1.0,0.0,4.5,-2.8,0.0,0.0,0.3,2.5,-2.2,19.8,0.8,-0.4,6.4,-1.3,0.0,0.0,0.7,2.7,14.2,37.9,-0.2,-0.6,4.4,0.0,0.0,0.0,0.5,0.2,24.7,26.2,-1.0,-5.3,1.8,0.0,0.0,0.0,-0.5,-2.5,-8.0,-28.5,0.5,0.0,1.7,-2.7,0.0,0.0,-0.2,1.0,-9.4,-1.2,0.4,0.0,4.9,0.0,0.0,0.0,0.6,1.4,31.3,42.7,-0.8,0.0,0.7,-3.8,6.5,0.0,0.3,-3.3,18.7,-13.6,-0.9,0.0,2.2,-4.1,7.4,0.0,0.5,-2.4,20.9,-2.6,0.0,0.0,5.8,-4.1,4.0,-0.5,0.4,0.3,20.4,23.3,0.7,0.0,10.0,-5.7,0.0,0.0,0.5,2.2,-3.0,20.7,1.3,0.0,11.1,-3.4,0.0,0.0,0.4,3.4,11.5,48.2,0.9,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,10
55,0,175,94,100,202,380,179,143,28,11,-5,20,?,71,0,72,20,0,0,48,0,0,0,0,0,0,0,64,36,0,0,36,0,0,0,0,0,0,20,52,48,0,0,56,0,0,0,0,0,0,64,32,0,0,0,72,0,0,0,0,0,0,0,60,12,0,0,44,0,0,0,0,0,0,0,60,44,0,0,32,0,0,0,0,0,0,56,0,0,0,0,0,0,0,0,0,0,0,0,40,44,0,0,20,0,0,0,0,0,0,0,52,40,0,0,32,0,0,0,0,0,0,0,56,48,0,0,36,0,0,0,0,0,0,0,60,48,0,0,36,0,0,0,0,0,0,0,64,40,0,0,40,0,0,0,0,0,0,0.9,0.0,7.8,-0.7,0.0,0.0,1.1,1.9,27.3,45.1,0.1,0.0,9.1,-2.6,0.0,0.0,0.4,1.5,24.5,36.8,-0.4,-0.4,1.6,-2.2,0.0,0.0,-1.0,-0.9,-1.5,-9.2,-0.4,-8.2,1.8,0.0,0.0,0.0,-0.7,-1.7,-23.4,-35.6,0.9,0.0,3.2,-0.4,0.0,0.0,0.7,1.2,9.4,18.0,-0.1,0.0,5.1,-2.5,0.0,0.0,0.3,0.6,9.8,12.6,1.6,-6.5,0.0,0.0,0.0,0.0,-0.4,-0.4,-18.2,-22.4,2.1,0.0,1.2,-6.9,0.0,0.0,-0.5,2.9,-12.7,18.0,0.7,0.0,9.0,-7.9,0.0,0.0,0.1,4.1,7.6,51.0,0.4,0.0,15.0,-5.5,0.0,0.0,0.1,3.3,28.8,63.1,0.1,0.0,15.2,-3.7,0.0,0.0,0.6,3.0,36.8,68.0,0.1,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,1
75,0,190,80,88,181,360,177,103,-16,13,61,3,?,?,0,48,40,0,0,28,0,0,0,0,0,0,0,40,24,0,0,24,0,0,0,0,0,0